In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Iris.csv')
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
data['Species'] = data['Species'].str.strip().map(mapping)

x_train = data.drop(['Id', 'Species'], axis=1).values
y_train = data['Species'].values

length = len(y_train)
print(f"Length of the data: {length}")

shape = x_train.shape
print(f"Shape of the data: {shape}")

number_of_classes = 3
print(f"Number of classes: {number_of_classes}")

x_features = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
number_of_features = len(x_features)
print(f"Number of features: {number_of_features}")

data.head()

Length of the data: 150
Shape of the data: (150, 4)
Number of classes: 3
Number of features: 4


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [4]:
def one_hot_encoding(y, m, n):
    y_one_train = np.zeros((m, n))
    y_one_train[np.arange(m), y] = 1
    return y_one_train

In [5]:
y_one_train = one_hot_encoding(y_train, length, number_of_classes)

In [6]:
def initialize_weights(input, hidden, output):
    w1 = np.ones((input, hidden))
    b1 = np.ones((1, hidden))
    w2 = np.ones((hidden, output))
    b2 = np.ones((1, output))
    return w1, b1, w2, b2

In [7]:
input_size = 4
hidden_size = 5
output_size = 3

In [8]:
w1, b1, w2, b2 = initialize_weights(input_size, hidden_size, output_size)
print(w1)
print(b2)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[[1. 1. 1.]]


In [9]:
def relu(z):
    return np.maximum(0, z)

In [10]:
def relu_derivative(z):
    return (z > 0).astype(float)